In [9]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import os
from env import get_db_url
# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format


In [10]:
def get_mall_data():
    filename = 'mall_data.csv'

    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    else:
        df = pd.read_sql(
            '''
            SELECT *
            FROM customers; 
            '''
            ,
            get_db_url('mall_customers')
        )

        df.to_csv(filename)

        return df

In [12]:
df = get_mall_data()

In [13]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

add_upper_outlier_columns(df, k=1.5)

df.head()


,customer_id,gender,age,annual_income,spending_score,customer_id_outliers,age_outliers,annual_income_outliers,spending_score_outliers
0,1,Male,19,15,39,0,0,0.00,0
1,2,Male,21,15,81,0,0,0.00,0
2,3,Female,20,16,6,0,0,0.00,0
3,4,Female,23,16,77,0,0,0.00,0
4,5,Female,31,17,40,0,0,0.00,0


In [14]:
df.shape

(200, 9)

In [15]:
df.isna().sum()

customer_id                0
gender                     0
age                        0
annual_income              0
spending_score             0
customer_id_outliers       0
age_outliers               0
annual_income_outliers     0
spending_score_outliers    0
dtype: int64